In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter

import dsds.transform as t
import dsds.fs as fs

from sklearn.preprocessing import power_transform
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 100, n_informative = 60, n_redundant = 40)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,3.058149,-1.435563,-8.457014,1.640523,0.57398,1.582525,2.308277,13.221124,-7.143688,-36.438022,3.814431,-4.75125,-3.189714,9.806908,-0.183783,16.859076,10.27535,5.147062,-3.443349,-8.584539,-15.966414,2.123187,8.214296,-0.93946,-0.778414,-2.010766,17.859377,-7.816131,21.152876,-1.586448,-0.618222,18.445602,2.247117,2.515964,-0.349371,1.054176,…,9.948941,-0.895045,2.468667,-22.502332,-3.958227,-7.982236,-0.953012,0.585046,-2.062462,2.423506,-20.285802,-1.004726,4.286519,5.889308,-4.124951,-26.749275,-1.134209,2.119505,-0.629102,-2.66286,-14.224477,3.415301,0.999277,-4.772086,-0.554316,6.196093,3.530679,-5.114195,-1.358051,-0.178766,4.458194,-5.733936,-10.36318,2.492106,-5.063588,21.50818,13.968265
1,-4.240389,17.997809,1.144189,17.376292,-5.606116,-9.674666,-16.030109,21.391422,3.341636,22.684675,4.153427,-7.615007,3.25067,8.886227,0.943289,10.528976,26.067408,4.718593,1.777174,-0.285101,-40.637885,2.089022,-1.185221,5.081185,5.727927,-2.918525,31.685489,-17.507925,-10.800831,-1.276797,1.287781,-51.359098,-10.006753,-1.504763,3.232843,-0.070627,…,16.886958,-1.857979,-1.662503,25.22916,8.487965,9.040261,1.146433,-8.901604,16.067657,5.47278,-16.519791,-3.156294,3.752677,-80.718244,-4.705964,-17.946756,-2.540582,2.676143,4.549622,2.3248,16.10472,1.457568,1.097254,-21.770353,-15.844483,-2.146103,14.178725,7.550425,-8.874459,0.571526,-9.847451,-30.101083,-24.127427,-2.776225,4.185251,23.653563,-23.288075
1,11.928344,31.479023,1.063552,-4.621436,-2.458862,-4.94459,9.4182,38.8289,-1.867021,-19.590649,-1.670916,1.379978,-1.617054,6.212348,-0.463167,37.659407,0.428669,0.488047,2.197062,-9.2984,21.759085,2.382145,-12.093664,-4.374372,9.963876,-3.07839,15.341387,21.865564,2.285397,-0.898613,1.068417,12.952227,0.80591,-0.970488,6.370502,0.752032,…,-0.703537,-7.694953,9.216952,-49.822867,2.346322,-23.016954,-5.112293,-9.83143,-17.114596,1.437586,0.699386,4.955116,-4.640043,-5.489474,-3.709112,-12.71997,2.66036,0.142703,2.989991,0.153306,-33.791032,2.388159,-4.039793,50.727605,-1.268173,0.327628,-2.068343,-0.796484,5.927812,-6.620684,4.260836,-23.02917,18.811763,1.882932,-1.907203,14.100935,13.38047
1,3.2263,-2.524616,-9.169814,33.727827,-5.986883,-5.091103,4.748894,-10.473185,-4.315662,1.213426,-4.524933,-0.619277,1.078335,15.082621,2.203932,9.175053,11.092248,6.635371,0.144578,36.47562,-6.812973,6.588839,34.483382,11.026605,1.075992,-4.785699,-47.605731,9.607173,23.320423,-5.63941,-2.410583,-26.765943,7.83919,-1.134767,7.205256,-9.532363,…,22.456352,10.660007,-4.736875,32.500524,4.54275,-25.384756,0.470849,-5.784234,-0.300482,-2.93609,15.81511,-3.377828,11.791298,21.519033,-2.531647,37.564108,2.317877,5.941745,-1.701458,8.774312,26.370616,-6.752921,-0.654093,16.489194,0.179207,1.069075,3.809768,-0.855404,4.269719,6.164437,-2.741119,48.028981,-23.109113,-4.120229,3.220088,39.625175,24.358839
1,-1.883331,20.368442,0.325983,1.881899,4.485705,-1.143816,-5.998849,10.686322,-4.513072,4.224386,2.002327,-8.457721,-1.654616,-0.894476,4.904

## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Fscore
2. Mutual Information Score
3. MRMR feature selection strategies
4. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Fscore

In [4]:
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.03s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",0.015285,0.901607
"""column_1""",220.520101,7.8729e-50
"""column_2""",1059.895965,2.8111e-231
"""column_3""",36.568048,1.4795e-9
"""column_4""",1224.560847,1.1620e-266
"""column_5""",0.63521,0.425453
"""column_6""",1083.866996,1.9680e-236
"""column_7""",81.838032,1.5026e-19
"""column_8""",1325.118046,3.0723e-288


In [5]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.14s in computing Fscore.


,feature,f_value,p_value
0,column_0,0.015285,9.016068e-01
1,column_1,220.520101,7.872924e-50
2,column_2,1059.895965,2.811065e-231
3,column_3,36.568048,1.479489e-09
4,column_4,1224.560847,1.161997e-266
5,column_5,0.635210,4.254527e-01
6,column_6,1083.866996,1.968046e-236
7,column_7,81.838032,1.502625e-19
8,column_8,1325.118046,3.072287e-288
9,column_9,288.011404,1.661015e-64


In [6]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.05s in computing Fscore.


,feature,f_value,p_value
0,column_0,0.015285,9.016068e-01
1,column_1,220.520101,7.872924e-50
2,column_2,1059.895965,2.811065e-231
3,column_3,36.568048,1.479489e-09
4,column_4,1224.560847,1.161997e-266
5,column_5,0.635210,4.254527e-01
6,column_6,1083.866996,1.968046e-236
7,column_7,81.838032,1.502625e-19
8,column_8,1325.118046,3.072287e-288
9,column_9,288.011404,1.661015e-64


# Mutual Information Score

In [7]:
# Took 16s
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 100/100 [00:07<00:00, 13.23it/s]


feature,estimated_mi
str,f64
"""column_88""",0.027004
"""column_38""",0.026539
"""column_61""",0.023837
"""column_87""",0.023037
"""column_70""",0.023024
"""column_13""",0.014162
"""column_58""",0.013852
"""column_46""",0.012389
"""column_83""",0.010688


In [8]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [9]:
# Took 1m38s seconds by using sklearn. The reason is that sklearn's implementation did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality.
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_88""",0.027004
"""column_38""",0.026539
"""column_61""",0.023837
"""column_87""",0.023035
"""column_70""",0.023024
"""column_13""",0.014162
"""column_58""",0.013852
"""column_46""",0.012389
"""column_83""",0.010688


# MRMR Feature selection Strategy

In [10]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [11]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [12]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [00:11<00:00,  4.44it/s]

Spent 15.02s to compute mrmr.


['column_38',
 'column_87',
 'column_50',
 'column_82',
 'column_88',
 'column_61',
 'column_70',
 'column_81',
 'column_64',
 'column_56',
 'column_18',
 'column_97',
 'column_91',
 'column_60',
 'column_46',
 'column_13',
 'column_65',
 'column_23',
 'column_55',
 'column_54',
 'column_4',
 'column_39',
 'column_58',
 'column_8',
 'column_41',
 'column_33',
 'column_69',
 'column_37',
 'column_83',
 'column_6',
 'column_59',
 'column_2',
 'column_96',
 'column_52',
 'column_44',
 'column_63',
 'column_15',
 'column_71',
 'column_68',
 'column_19',
 'column_73',
 'column_10',
 'column_76',
 'column_57',
 'column_9',
 'column_1',
 'column_36',
 'column_16',
 'column_26',
 'column_28']

In [13]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:00<00:00, 210.92it/s]

Output is sorted in order of selection (relevance).
Spent 0.28s in computing.


['column_38',
 'column_87',
 'column_50',
 'column_82',
 'column_88',
 'column_61',
 'column_70',
 'column_81',
 'column_64',
 'column_56',
 'column_18',
 'column_97',
 'column_91',
 'column_60',
 'column_46',
 'column_13',
 'column_65',
 'column_23',
 'column_55',
 'column_54',
 'column_4',
 'column_39',
 'column_58',
 'column_8',
 'column_41',
 'column_33',
 'column_69',
 'column_37',
 'column_83',
 'column_6',
 'column_59',
 'column_2',
 'column_96',
 'column_52',
 'column_44',
 'column_63',
 'column_15',
 'column_71',
 'column_68',
 'column_19',
 'column_73',
 'column_10',
 'column_76',
 'column_57',
 'column_9',
 'column_1',
 'column_36',
 'column_16',
 'column_26',
 'column_28']

In [14]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:01<00:00, 43.24it/s]

Output is sorted in order of selection (relevance).
Spent 1.18s in computing.


['column_38',
 'column_87',
 'column_50',
 'column_82',
 'column_88',
 'column_61',
 'column_70',
 'column_81',
 'column_64',
 'column_56',
 'column_18',
 'column_97',
 'column_91',
 'column_60',
 'column_46',
 'column_13',
 'column_65',
 'column_23',
 'column_55',
 'column_54',
 'column_4',
 'column_39',
 'column_58',
 'column_8',
 'column_41',
 'column_33',
 'column_69',
 'column_37',
 'column_83',
 'column_6',
 'column_59',
 'column_2',
 'column_96',
 'column_52',
 'column_44',
 'column_63',
 'column_15',
 'column_71',
 'column_68',
 'column_19',
 'column_73',
 'column_10',
 'column_76',
 'column_57',
 'column_9',
 'column_1',
 'column_36',
 'column_16',
 'column_26',
 'column_28']

# Power Transform

In [15]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, target = target, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Inferring best paramters: 100%|██████████| 100/100 [00:01<00:00, 55.68it/s]


Spent 2.01s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,3.106381,-1.436369,-8.433941,1.63969,0.573507,1.588182,2.315861,13.228537,-7.081721,-37.137027,3.807188,-4.783764,-3.186472,9.80708,-0.183808,16.715579,10.341157,5.073999,-3.466679,-8.566278,-15.896711,2.125785,8.228512,-0.944099,-0.780662,-2.015953,18.034748,-7.686919,21.144807,-1.590185,-0.618149,18.355608,2.246408,2.499876,-0.348516,1.055028,…,9.982846,-0.891643,2.497802,-22.412686,-3.989805,-8.020299,-0.953677,0.584614,-2.048009,2.401046,-20.623881,-1.00331,4.25745,5.885079,-4.126242,-25.787959,-1.137396,2.121766,-0.629173,-2.63275,-13.815664,3.415407,0.996592,-4.774213,-0.555119,6.149123,3.510069,-5.123808,-1.348525,-0.178819,4.424162,-5.714436,-10.320223,2.471547,-5.132027,21.204696,14.008619
1,-4.160517,17.956153,1.145077,17.343721,-5.629378,-9.57377,-15.882911,21.405992,3.360712,22.323281,4.145134,-7.682048,3.254017,8.886376,0.94275,10.455728,26.311878,4.654771,1.769461,-0.285049,-40.388959,2.091549,-1.184615,5.002912,5.663458,-2.928181,32.070434,-17.094739,-10.803946,-1.279355,1.288058,-51.718728,-10.014132,-1.511523,3.278013,-0.070622,…,16.959372,-1.845704,-1.647775,25.33447,8.387104,8.994727,1.14551,-8.942883,16.415835,5.390023,-16.773487,-3.14626,3.729144,-80.877809,-4.707554,-17.390472,-2.553351,2.679481,4.547482,2.349146,16.608687,1.457594,1.094082,-21.79001,-16.040002,-2.154727,14.010721,7.53304,-8.679236,0.571035,-9.962124,-29.890805,-23.985929,-2.801061,4.135259,23.307814,-23.205955
1,12.320436,31.389485,1.064331,-4.626028,-2.465068,-4.907982,9.487293,38.861515,-1.859812,-19.890947,-1.672759,1.375816,-1.616016,6.212435,-0.463312,37.230481,0.42898,0.486783,2.186011,-9.277851,21.866928,2.385296,-12.068666,-4.437371,9.816181,-3.088926,15.482983,22.443878,2.285121,-0.899979,1.068616,12.897608,0.805792,-0.973598,6.502208,0.752494,…,-0.703113,-7.576241,9.447909,-49.563216,2.332979,-23.190149,-5.123462,-9.879218,-16.743198,1.428355,0.697879,4.975785,-4.673247,-5.493272,-3.710202,-12.377554,2.646663,0.142719,2.9889,0.153468,-32.45197,2.388218,-4.068567,50.666636,-1.271787,0.327342,-2.076923,-0.796896,6.036633,-6.652067,4.229147,-22.883009,18.912765,1.870096,-1.921104,13.932734,13.418422
1,3.278936,-2.526723,-9.143806,33.647163,-6.01262,-5.052795,4.773311,-10.467894,-4.287159,1.209631,-4.53445,-0.620261,1.078845,15.08294,2.201566,9.115179,11.165818,6.527572,0.144505,36.614765,-6.792857,6.604777,34.590417,10.776886,1.071974,-4.807025,-46.954516,9.786177,23.311196,-5.669174,-2.409771,-26.917004,7.834052,-1.138882,7.364087,-9.501773,…,22.563958,10.855851,-4.654882,32.648852,4.503865,-25.58279,0.470668,-5.805776,-0.300045,-2.967299,15.580503,-3.366628,11.657679,21.49129,-2.532232,39.146454,2.306983,5.953631,-1.701882,8.972562,27.370729,-6.752613,-0.655337,16.475853,0.179115,1.066514,3.786487,-0.855874,4.334901,6.13644,-2.756788,48.423225,-22.975723,-4.167299,3.187222,38.933739,24.446514
1,-1.862192,20.318968,0.32607,1.880845,4.469342,-1.140627,-5.964012,10.691774,-4.482491,4.193736,1.99982,-8.53615,-1.653536,-0.89

In [17]:
# Sklearn with Pandas

# In this case, the benefits of using dsds are:
# 1. Much better performance.
# 2. Users need to do extra steps to put data back in dataframe format.
# 3. Dsds can save the plan using LazyFrame.blueprint.preserve_as_pickle()... But in Pandas,
# you have to use the PowerTransformer class, which makes code longer and uglier (and less organized)

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 7.84s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0,3.106381,-1.436369,-8.433941,1.639690,0.573507,1.588182,2.315861,13.228537,-7.081721,...,-5.123808,-1.348525,-0.178819,4.424162,-5.714436,-10.320223,2.471547,-5.132027,21.204696,14.008619
1,1,-4.160517,17.956153,1.145077,17.343721,-5.629378,-9.573770,-15.882911,21.405992,3.360712,...,7.533040,-8.679236,0.571035,-9.962124,-29.890805,-23.985929,-2.801061,4.135258,23.307814,-23.205955
2,1,12.320436,31.389485,1.064331,-4.626028,-2.465068,-4.907982,9.487293,38.861515,-1.859812,...,-0.796896,6.036633,-6.652067,4.229147,-22.883009,18.912765,1.870096,-1.921104,13.932734,13.418422
3,1,3.278936,-2.526723,-9.143806,33.647163,-6.012620,-5.052795,4.773311,-10.467894,-4.287159,...,-0.855874,4.334901,6.136440,-2.756788,48.423225,-22.975723,-4.167299,3.187222,38.933739,24.446514
4,1,-1.862192,20.318968,0.326070,1.880845,4.469342,-1.140627,-5.964012,10.691774,-4.482491,...,-2.340471,6.776495,-1.579208,-9.621510,32.640783,11.077348,1.921243,5.547765,36.752247,-5.130302


# GC

In [18]:
import gc 
gc.collect()

1039